## For daily income maximization and income per work hours maximization

In [ ]:
# from holidays.countries.united_states import US
# import holidays
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan, when, count, col, lit, udf
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.context import SparkContext
import pandas as pd 
import numpy as np 
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime
from dateutil.relativedelta import relativedelta
from functools import reduce
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Bucketizer
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator, RegressionEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import GBTRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import partial_dependence, plot_partial_dependence, permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import ssl
from urllib import request
from PIL import Image

spark = SparkSession.builder.appName("tripDataCSVLoad").getOrCreate()

In [ ]:
df_trip_fm =spark.read.parquet("spark-warehouse/trip_driver_fmax")
print(df_trip_fm.count())
print(df_trip_fm.schema.names)

## Check suspicious taxi rotation activity

In [ ]:
### number of taxis driven per day

df_taxi_driven_per_day_check = df_trip_fm.groupBy('driver_id','pickup_date','taxi_id').count()\
    .groupBy('driver_id','pickup_date').count().sort(F.desc('count')).toPandas()

# df_taxi_driven_per_day_check.groupby(by='count',as_index=False)['driver_id'].count().sort_values(by='count',ascending=0)

#----------------------------------------------------------------------------------------------
# Remove suspicious data of driver_id - pickup_date combo occupying more than 2 taxis in a day
taxi_per_day_limit = 2
df_combo_to_drop = df_taxi_driven_per_day_check.iloc[np.where(df_taxi_driven_per_day_check['count']>taxi_per_day_limit)].drop(columns='count',axis=1)

#taxis occupied by driver in a day	driver-days
# 18	1
# 17	1
# 16	1
# 15	2
# 13	2
# 12	2
# 11	2
# 10	1
# 9	    5
# 8	    7
# 7	    11
# 6	    2
# 5	    17
# 4	    22
# 3	    199
# 2	    26248
# 1	    688944
#------------------------------------------------------------------------------------------------



## Remove suspicious Driver ID - Pickup Date

In [ ]:
print(df_trip_fm.count())
df_combo_to_drop_spark =spark.createDataFrame(df_combo_to_drop) 

joining_conds = [df_trip_fm.driver_id == df_combo_to_drop_spark.driver_id,
                 df_trip_fm.pickup_date == df_combo_to_drop_spark.pickup_date]
df_trip_fm = df_trip_fm.join(df_combo_to_drop_spark, joining_conds, 'leftanti')
print(df_trip_fm.count())

## aggregate to arrive at taxi_driver_id - pickup_date level

In [ ]:
### check pickup_area distribution

total_trips = df_trip_fm.count()
df_pickup_area = df_trip_fm.groupBy('pickup_area').count().toPandas().sort_values(by='count',ascending=0)
df_pickup_area['percentage'] = df_pickup_area['count'].apply(lambda x: round(float(x)/total_trips,2))
df_pickup_area_filtered = df_pickup_area.iloc[np.where(df_pickup_area['percentage']>=0.01)]
df_pickup_area_filtered['abbrv_pickup_area'] = df_pickup_area_filtered['pickup_area'].apply(lambda x: re.split(', ',x)[0])

pickup_area_dict = df_pickup_area_filtered[['pickup_area','abbrv_pickup_area']].set_index('pickup_area').to_dict()['abbrv_pickup_area']

pickup_area_dict

In [ ]:
### Add flag weekend, driver_revenue, pickup_cat (pickup_period - pickup_area)
getabbrv = udf(lambda x: 'others' if x not in pickup_area_dict.keys() else pickup_area_dict[x],StringType())

flgweekday = udf(lambda x: 0 if x in [0,6] else 1,IntegerType())
df_trip_fm = df_trip_fm.withColumn('flg_weekday',flgweekday(F.dayofweek(F.to_timestamp(col('pickup_datetime')))))\
    .withColumn('driver_revenue',col('fare_amount')+col('tip_amount'))\
        .withColumn('pickup_cat',F.concat_ws('_',col('pickup_period'),getabbrv(col('pickup_area'))))

In [ ]:
### aggregat numeric 

df_daily_agg_numeric = df_trip_fm.groupBy('driver_id','pickup_date')\
    .agg(F.round(F.sum(col('trip_time_in_secs'))/3600.0,2).alias('hours'),
    F.count(col('trip_id')).alias('trips'),
    F.round(F.sum(col('driver_revenue')),2).alias('revenue'))\
        .withColumn('revenue_per_hour',F.round(col('revenue')/col('hours'),2))\
            .withColumn('minutes_per_trip',F.round(col('hours')*60.0/col('trips'),2))

df_daily_agg_numeric.show(5)

In [ ]:
### aggregate by trip

simplify_name = udf(lambda x: str(x).lower().replace('.','_').replace('-<','lt').replace('-','to'),StringType())

df_trip_pickup_cat = df_trip_fm.groupBy('driver_id','pickup_date','pickup_cat')\
    .agg(F.count(col('trip_id')).alias('trips')).\
        withColumn('pickup_cat',simplify_name(col('pickup_cat'))).toPandas()

df_driver_date_tot_trip = df_trip_pickup_cat.groupby(by=['driver_id','pickup_date'],as_index=False)['trips'].sum()

index_cols = ['driver_id','pickup_date']

print(df_trip_pickup_cat.shape)
df_trip_pickup_cat = df_trip_pickup_cat.merge(df_driver_date_tot_trip.rename(columns={'trips':'tot_trips'}),
                                             on=index_cols,
                                             how='left')
print(df_trip_pickup_cat.shape)
df_trip_pickup_cat['percentage'] = df_trip_pickup_cat.apply(lambda row: round(float(row['trips'])/row['tot_trips'],2),axis=1)

df_trip_pickup_cat_tp = pd.pivot_table(df_trip_pickup_cat,
                                        index=index_cols,
                                        columns=['pickup_cat'],
                                        values=['percentage']).fillna(0.0).reset_index()

df_trip_pickup_cat_tp.columns = [v[0] if i<len(index_cols) else v[1] for i,v in enumerate(df_trip_pickup_cat_tp.columns.values)]

df_trip_pickup_cat_spark = spark.createDataFrame(df_trip_pickup_cat_tp)
df_trip_pickup_cat_spark.printSchema()

In [ ]:
### no. of taxi this driver occupied in a day

df_taxi_per_day = spark.createDataFrame(df_taxi_driven_per_day_check.rename(columns={'count':'taxi_per_day'}))
df_taxi_per_day.printSchema()

## Join all features to predict Daily Revenue

In [ ]:
joining_keys = ['driver_id','pickup_date']
print(df_daily_agg_numeric.count())
df_driver_abt= df_daily_agg_numeric.join(df_trip_pickup_cat_spark, joining_keys, 'left')
print(df_driver_abt.count())
df_driver_abt= df_driver_abt.join(df_taxi_per_day, joining_keys, 'left')
print(df_driver_abt.count())
df_driver_abt.printSchema()

In [ ]:
### Avg daily revenue

print(f"avg daily revenue of Driver: {round(df_driver_abt_pd['revenue'].mean(),2)}")

In [ ]:
### Convert to Pandas

df_driver_abt_pd = df_driver_abt.toPandas()
df_driver_abt_pd.shape

### Set features
non_feature_cols = ['driver_id','pickup_date','hours','revenue','revenue_per_hour']
features_input = [i for i in df_driver_abt.columns if i not in non_feature_cols]
features_input

## Maximising Daily Revenue for Driver

In [ ]:
# Create Training and Test Split
target = 'revenue'
X_train, X_test, y_train, y_test = train_test_split(df_driver_abt_pd[features_input], 
                                                    df_driver_abt_pd[target], 
                                                    random_state=9000, 
                                                    test_size=0.3)

# Hyperparameters for GradientBoostingRegressor

param_grid = {'n_estimators': [1000],
          'max_depth': [3],
          'min_samples_split': [5],
          'learning_rate': [0.01],
          'loss': ['ls']}

# Create an instance of gradient boosting regressor

gbr = GradientBoostingRegressor()

gbr_grid = GridSearchCV(estimator = gbr, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 1)
# Fit the model

gbr_grid.fit(X_train, y_train)

# Print Coefficient of determination R^2

print("Model Accuracy: {:.3f}".format(gbr_grid.score(X_test, y_test)))

# Create the mean squared error

mse = mean_squared_error(y_test, gbr_grid.predict(X_test), squared=False)
print("Root mean squared error (RMSE) on test set: {:.2f}".format(mse))

In [ ]:
# Get Feature importance data using feature_importances_ attribute

feature_importance = gbr_grid.best_estimator_.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(8, 8))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(X_test.columns.values)[sorted_idx])
plt.title('Daily Revenue Maximization: Feature Importance')
result = permutation_importance(gbr_grid, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(15,5))
for ax,feat in zip(axes,np.array(X_test.columns.values)[sorted_idx][-5:]):
    print(ax)
    print(feat)
    plot_partial_dependence(gbr_grid,X_test,
    [feat],
    subsample=5,
    n_jobs=-1,
    grid_resolution=20,
    ax=ax)
plt.show()

In [ ]:
### Save model
gbr_grid.save('models/driver_daily_revenue_predictor')

## Maximizing revenue_per_hour

In [ ]:
### Remove outliers who work 0.0 hours
df_driver_abt_rph = df_driver_abt_pd.iloc[np.where(df_driver_abt_pd['revenue_per_hour'].notnull())]

In [ ]:
### impute extreme revenue per hour

rph_cutoff = np.percentile(df_driver_abt_rph['revenue_per_hour'],[0,1,5,25,50,75,95,99,100])[-2]

df_driver_abt_rph['revenue_per_hour'] = np.where(df_driver_abt_rph['revenue_per_hour']>rph_cutoff,
                                                rph_cutoff,
                                                df_driver_abt_rph['revenue_per_hour'])

### check daily hours

np.percentile(df_driver_abt_rph['hours'],[0,1,5,25,50,75,95,99,100])

### drop those who work less than 1 hour
print(df_driver_abt_rph.shape)
df_driver_abt_rph_fltrd = df_driver_abt_rph.iloc[np.where(df_driver_abt_rph['hours']>=1)].reset_index(drop=True)
print(df_driver_abt_rph_fltrd.shape)

In [ ]:
### Set features
non_feature_cols = ['driver_id','pickup_date','revenue','revenue_per_hour','trips','minutes_per_trip']
features_input = [i for i in df_driver_abt_rph_fltrd.columns if i not in non_feature_cols]

# Create Training and Test Split
target = 'revenue_per_hour'
X_train, X_test, y_train, y_test = train_test_split(df_driver_abt_rph_fltrd[features_input], 
                                                    df_driver_abt_rph_fltrd[target], 
                                                    random_state=9000, 
                                                    test_size=0.3)

# Hyperparameters for GradientBoostingRegressor

param_grid = {'n_estimators': [1000],
          'max_depth': [3],
          'min_samples_split': [5],
          'learning_rate': [0.01],
          'loss': ['ls']}

# Create an instance of gradient boosting regressor

gbr = GradientBoostingRegressor()

gbr_grid_rph = GridSearchCV(estimator = gbr, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 1)
# Fit the model

gbr_grid_rph.fit(X_train, y_train)

# Print Coefficient of determination R^2

print("Model Accuracy: {:.3f}".format(gbr_grid_rph.score(X_test, y_test)))

# Create the mean squared error

mse = mean_squared_error(y_test, gbr_grid_rph.predict(X_test), squared=False)
print("Root mean squared error (RMSE) on test set: {:.2f}".format(mse))

In [ ]:
# Get Feature importance data using feature_importances_ attribute

feature_importance = gbr_grid_rph.best_estimator_.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(8, 8))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(X_test.columns.values)[sorted_idx])
plt.title('Revenue Per Hour Maximization: Feature Importance')
result = permutation_importance(gbr_grid_rph, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(15,5))
for ax,feat in zip(axes,np.array(X_test.columns.values)[sorted_idx][-5:]):
    print(ax)
    print(feat)
    plot_partial_dependence(gbr_grid_rph,X_test,
    [feat],
    subsample=5,
    n_jobs=-1,
    grid_resolution=20,
    ax=ax)
plt.show()

In [ ]:
### Save model
gbr_grid_rph.save('models/driver_daily_rev_per_hour_predictor')

## Taxi - Daily level

In [ ]:
### aggregat numeric 

df_taxi_daily_agg_numeric = df_trip_fm.groupBy('taxi_id','pickup_date')\
    .agg(F.round(F.sum(col('trip_time_in_secs'))/3600.0,2).alias('hours'),
    F.count(col('trip_id')).alias('trips'),
    F.round(F.sum(col('driver_revenue')),2).alias('revenue'))\
        .withColumn('revenue_per_hour',F.round(col('revenue')/col('hours'),2))\
            .withColumn('minutes_per_trip',F.round(col('hours')*60.0/col('trips'),2))

df_taxi_daily_agg_numeric.show(5)

In [ ]:
### number of drivers ocuupying taxi per day

df_driver_occ_taxi_per_day_check = df_trip_fm.groupBy('taxi_id','pickup_date','driver_id').count()\
    .groupBy('taxi_id','pickup_date').count().sort(F.desc('count')).toPandas()

# df_driver_occ_taxi_per_day_check.groupby(by='count',as_index=False)['taxi_id'].count().sort_values(by='count',ascending=0)

#----------------------------------------------------------------------------------------------
# Remove suspicious data of taxi_id - pickup_date combo occupied by more than 3 drivers in a day
drivers_per_day_limit = 3
df_combo_taxi_to_drop = df_driver_occ_taxi_per_day_check.iloc[np.where(df_driver_occ_taxi_per_day_check['count']>drivers_per_day_limit)].drop(columns='count',axis=1)

#drivers occupying taxi in a day	taxi-days
#	24	1
#	13	1
#	5	2
#	4	484
#	3	53320
#	2	248957
#	1	81583
#------------------------------------------------------------------------------------------------



In [ ]:
df_driver_occ_taxi_per_day_check.groupby(by='count',as_index=False)['taxi_id'].count().rename(columns={'taxi_id':'no of taxi-day','count':'drivers occupying in a day'})

In [ ]:
print(df_trip_fm.count())
df_combo_taxi_to_drop_spark =spark.createDataFrame(df_combo_taxi_to_drop) 

joining_conds = [df_trip_fm.taxi_id == df_combo_taxi_to_drop_spark.taxi_id,
                 df_trip_fm.pickup_date == df_combo_taxi_to_drop_spark.pickup_date]
df_trip_fm = df_trip_fm.join(df_combo_taxi_to_drop_spark, joining_conds, 'leftanti')
print(df_trip_fm.count())

In [ ]:
### no. of drivers taxi was occupied by in a day

df_driver_per_day = spark.createDataFrame(df_driver_occ_taxi_per_day_check.rename(columns={'count':'driver_per_day'}))
df_driver_per_day.printSchema()

In [ ]:
joining_keys = ['taxi_id','pickup_date']
print(df_taxi_daily_agg_numeric.count())
df_taxi_abt= df_taxi_daily_agg_numeric.join(df_driver_per_day, joining_keys, 'left')
print(df_taxi_abt.count())

In [ ]:
### check correlation between revenue, hours and no. of drivers occupying taxi per day
vector_col = "corr_features"
cols_to_check_corr = ['revenue','hours','driver_per_day']
df = df_taxi_abt.select(*[cols_to_check_corr]).toPandas()
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

## Find example case of taxi occupied by multiple drivers in a day

In [ ]:
# df_driver_occ_taxi_per_day_check.iloc[np.where(df_driver_occ_taxi_per_day_check['count']==3)]

# taxi_id	pickup_date	count
# 8892	2013-04-28	3
# 11086	2013-04-06	3

df_trip_fm.filter((col('taxi_id')==8892)&(col('pickup_date')=='2013-04-28')).sort(col('pickup_datetime').asc())\
    .select('taxi_id','driver_id','pickup_datetime','trip_time_in_secs').show(50)